# 10. Approximate Inference [In Progress]

---
## 10.1. Variational Inference

### 10.1.1 Factorized distributions

### 10.1.2 Properties of factorized approximations

### 10.1.3 Example: The univariate Gaussian

### 10.1.4 Model comparison

---
## 10.2. Illustration: Variational Mixture of Gaussians

### 10.2.1 Variational distribution

### 10.2.2 Variational lower bound

### 10.2.3 Predictive density

### 10.2.4 Determining the number of components

### 10.2.5 Induced factorizations

---
## 10.3. Variational Linear Regression

### 10.3.1 Variational distribution

### 10.3.2 Predictive distribution

### 10.3.3 Lower bound

---
## 10.4. Exponential Family Distributions

### 10.4.1 Variational message passing

---
## 10.5. Local Variational Methods

---
## 10.6. Variational Logistic Regression

### 10.6.1 Variational posterior distribution

### 10.6.2 Optimizing the variational parameters

### 10.6.3 Inference of hyperparameters

---
## 10.7. Expectation Propagation

### 10.7.1 Example: The clutter problem

### 10.7.2 Expectation propagation on graphs

Problem Setup

Given the following setup:

- I.I.D. Observed data: $\mathcal{X} = \left(X^{(1)}, \cdots, X^{(D)}\right)$, where each $X^{(i)} = (x^{(i)}_{1}, x^{(i)}_{2}, \cdots, x^{(i)}_{M})$
- Unobserved latent data or missing values: $\mathcal{Z}$
- Vector of unknown parameters: $\boldsymbol {\theta }$
- Likelihood function: $L({\boldsymbol {\theta }};\mathcal{X} ,\mathcal{Z} )=p(\mathcal{X} ,\mathcal{Z} \mid {\boldsymbol {\theta }})$

we have 2 tasks that are of interest:

- 1. Find the MLE ($\boldsymbol {\theta}$ is parameter) / MAP ($\boldsymbol {\theta}$ is random variable) estimates of the model parameters $\boldsymbol {\theta_{\rm{max}}}$ {cite}`Expectat45:online`:

\begin{align}
    \boldsymbol {\theta_{\rm{max}}} &= \underset{\boldsymbol {\theta}}{\operatorname{argmax}} \log p(\mathcal{X} \mid \boldsymbol{\theta }) \\
    &= \underset{\boldsymbol {\theta}}{\operatorname{argmax}} \log \sum_{z \in \mathcal{Z}} p(\mathcal{X} ,\mathcal{Z} = z \mid {\boldsymbol {\theta }}) \\
\end{align}

where $p(\mathcal{X} \mid \boldsymbol{\theta })$ is known as the ***Evidence / Marginal Likelihood / Incomplete-data Likelihood*** and $p(\mathcal{X} ,\mathcal{Z} \mid {\boldsymbol {\theta }})$ is known as the ***Complete-data likelihood***.

- 2. Find the posterior over the latent variables $\mathcal{Z}$, $p(\mathcal{Z} \mid \mathcal{X}, \boldsymbol {\theta_{\rm{max}} })$ {cite}`SVIPartI61:online`:

\begin{align}
    p(\mathcal{Z} \mid \mathcal{X}, \boldsymbol {\theta_{\rm{max}} }) &= \frac{p(\mathcal{X}, \mathcal{Z} \mid \boldsymbol {\theta_{\rm{max}} })}{
\sum_{z \in \mathcal{Z}} p(\mathcal{X}, \mathcal{Z} = z \mid \boldsymbol {\theta_{\rm{max}} })} \\
\end{align}

```{note} Notice that $\sum_{z \in \mathcal{Z}}$ is a general summation to encompass all types of marginalization possible, i.e. can be a multidimensional summation $\sum_{z_1 \in Z_1}\sum_{z_2 \in Z_2}\cdots\sum_{z_M \in Z_M}$ or integral(s) if $z$ is continuous or even a mix of summation and integration over different latent variables of interest.
```

However, this $p(\mathcal{X} \mid \boldsymbol{\theta }) = \sum_{z \in \mathcal{Z}} p(\mathcal{X} ,\mathcal{Z} = z \mid {\boldsymbol {\theta }})\,d\mathcal{Z}$ is often intractable (e.g. if $\mathcal{Z}$  is a sequence of events, so that the number of values grows exponentially with the sequence length, the exact calculation of the sum will be extremely difficult). Let's instead try to find a lower bound for it by expanding it {cite}`SVIPartI61:online`:

\begin{align}
    \log p(\mathcal{X} \mid \boldsymbol{\theta }) 
    &= \log \sum_{z \in \mathcal{Z}} p(\mathcal{X} ,\mathcal{Z} = z \mid {\boldsymbol {\theta }}) \\
\end{align}

Using ideas from importance sampling, assume we have another variational distribution [approximate posterior distribution to $p({\mathcal{Z}} \mid {\mathcal{X}}, {\bf \theta})$)], $q({\mathcal{Z}} \mid {\bf \phi})$, where $q({\mathcal{Z}} \mid {\bf \phi}) > 0$ whenever $p({\mathcal{Z}}) = \sum_{x \in \mathcal{X}} p({\mathcal{X}} = x, {\mathcal{Z}} \mid {\bf \theta}) > 0$, and we rewite:

\begin{align}
    \log p(\mathcal{X} \mid \boldsymbol{\theta }) 
    &= \log \sum_{z \in \mathcal{Z}} p(\mathcal{X} ,\mathcal{Z} = z \mid {\boldsymbol {\theta }}) \frac{q({\mathcal{Z} = z} \mid {\bf \phi})}{q({\mathcal{Z} = z} \mid {\bf \phi})} \\
    &= \log \operatorname {E}_{q({\mathcal{Z} = z} \mid {\bf \phi})} \left[\frac{p(\mathcal{X} ,\mathcal{Z} = z \mid {\boldsymbol {\theta }})}{q({\mathcal{Z} = z} \mid {\bf \phi})} \right] \\
\end{align}

By Jensen's Inequality, given concave function $f(X)$ (e.g. $\log$), $f\operatorname {E}\left[X\right] \geq \operatorname {E}\left[f(X)\right]$ {cite}`Variatio28:online`:

\begin{align}
    \log p(\mathcal{X} \mid \boldsymbol{\theta }) 
    &= \log \operatorname {E}_{q({\mathcal{Z} = z} \mid {\bf \phi})} \left[\frac{p(\mathcal{X} ,\mathcal{Z} = z \mid {\boldsymbol {\theta }})}{q({\mathcal{Z} = z} \mid {\bf \phi})} \right] \\
    &\geq \operatorname {E}_{q({\mathcal{Z} = z} \mid {\bf \phi})} \left[\log\left(\frac{p(\mathcal{X} ,\mathcal{Z} = z \mid {\boldsymbol {\theta }})}{q({\mathcal{Z} = z} \mid {\bf \phi})}\right)\right] \\
    &= \operatorname {E}_{q({\mathcal{Z} = z} \mid {\bf \phi})} \left[\log p(\mathcal{X} ,\mathcal{Z} = z \mid {\boldsymbol {\theta }}) - \log q({\mathcal{Z} = z} \mid {\bf \phi})\right] \\
    &= \operatorname {E}_{q({\mathcal{Z} = z} \mid {\bf \phi})} \left[\log p(\mathcal{X} ,\mathcal{Z} = z \mid {\boldsymbol {\theta }})\right] - \operatorname {E}_{q({\mathcal{Z} = z} \mid {\bf \phi})} \left[\log q({\mathcal{Z} = z} \mid {\bf \phi})\right] \\
    &= \underbrace{\underbrace{\operatorname {E}_{q({\mathcal{Z} = z} \mid {\bf \phi})} \left[\log p(\mathcal{X} ,\mathcal{Z} = z \mid {\boldsymbol {\theta }})\right]}_{\text{Expected Complete-data Log Likelihood}} + \underbrace{\operatorname{H}\left[\log q({\mathcal{Z}} \mid {\bf \phi})\right]}_{\text{Entropy of Variational Dist.}}}_{\text{ELBO / Negative Variational Free Energy } \mathcal{L}(q({\mathcal{Z}}\mid {\bf \phi}))} \\
\end{align}

Hence, we get an ***Evidence Lower Bound (ELBO)*** (also known as the ***Negative Variational Free Energy***) on the $\log$ Evidence. Instead of an inequality, we can get an exact equality of the form below by deriving the ELBO from rearranging the KL Divergence from our variational distribution (approximate posterior of latent variables) $q({\mathcal{Z}} \mid {\bf \phi})$ to our actual posterior over latent variables $p({\mathcal{Z}} \mid {\mathcal{x}}, {\bf \theta})$:

Derivation from ${\rm KL}(q({\mathcal{Z}} \mid {\bf \phi}) \mid\mid p({\mathcal{Z}} \mid {\mathcal{x}}, {\bf \theta}))$:

\begin{align}
    {\rm KL}(q({\mathcal{Z}} \mid {\bf \phi}) \mid\mid p({\mathcal{Z}} \mid {\mathcal{x}}, {\bf \theta}))
    &= \operatorname{E}_{q({\mathcal{Z} = z} \mid {\bf \phi})}\left[\log\left(\frac{q({\mathcal{Z} = z} \mid {\bf \phi})}{p({\mathcal{Z} = z} \mid {\mathcal{x}}, {\bf \theta})}\right)\right] \\
    &= \operatorname{E}_{q({\mathcal{Z} = z} \mid {\bf \phi})}\left[\log q({\mathcal{Z} = z} \mid {\bf \phi})\right] - \operatorname{E}_{q({\mathcal{Z} = z} \mid {\bf \phi})}\left[\log p({\mathcal{Z} = z} \mid {\mathcal{x}}, {\bf \theta})\right] \\
    &= \operatorname{E}_{q({\mathcal{Z} = z} \mid {\bf \phi})}\left[\log q({\mathcal{Z} = z} \mid {\bf \phi})\right] - \operatorname{E}_{q({\mathcal{Z} = z} \mid {\bf \phi})}\left[\log p({\mathcal{Z} = z}, {\mathcal{x}} \mid {\bf \theta})\right] + \operatorname{E}_{q({\mathcal{Z} = z} \mid {\bf \phi})}\left[\log p({\mathcal{x}} \mid {\bf \theta})\right] \\
    &= -\left[\operatorname {E}_{q({\mathcal{Z} = z} \mid {\bf \phi})} \left[\log p(\mathcal{X} ,\mathcal{Z} = z \mid {\boldsymbol {\theta }})\right] + \operatorname{H}\left[\log q({\mathcal{Z}} \mid {\bf \phi})\right]\right] + \operatorname{E}_{q({\mathcal{Z} = z} \mid {\bf \phi})}\left[\log p({\mathcal{x}} \mid {\bf \theta})\right] \\
    &= -\mathcal{L}(q({\mathcal{Z} = z}\mid {\bf \phi})) + \log p({\mathcal{x}} \mid {\bf \theta}) \because \text{Expectation is over latent variables }{\mathcal{Z} = z}\text{, which is independent of }{\mathcal{x}} \\
\end{align}

\begin{align}
    \therefore \log p({\mathcal{x}} \mid {\bf \theta}) 
    &= \mathcal{L}(q({\mathcal{Z}} \mid {\bf \phi})) + {\rm KL}(q({\mathcal{Z}} \mid {\bf \phi}) \mid\mid p({\mathcal{Z}} \mid {\mathcal{x}}, {\bf \theta})) \\
\end{align}

Since $\log p({\mathcal{x}} \mid {\bf \theta})$ is a constant, maximizing our ELBO / Negative Variational Free Energy will be equivalent to minimizing the ${\rm KL}(q({\mathcal{Z}} \mid {\bf \phi}) \mid\mid p({\mathcal{Z}} \mid {\mathcal{x}}, {\bf \theta}))$ (0 when $q({\mathcal{Z}} \mid {\bf \phi}) = p({\mathcal{Z}} \mid {\mathcal{x}}, {\bf \theta})$), making our variational approximation as close as possible to the actual posterior over latents. After this procedure, our 2 tasks will look like:

- 1. Find the MLE (${\bf\theta}, {\bf\phi}$ are parameters) / MAP (${\bf\theta}, {\bf\phi}$ are random variables) estimates of the model parameters ${\bf \theta_{\rm{max}}}, {\bf \phi_{\rm{max}}}$ by maximizing the ELBO:

\begin{align}
    {\bf\theta_{\rm{max}}} &= \underset{\boldsymbol {\theta}}{\operatorname{argmax}} \log p(\mathcal{X} \mid \boldsymbol{\theta }) \\
    {\bf\theta_{\rm{max}}}, {\bf\phi_{\rm{max}}} &\approx \underset{{\bf \theta}, {\bf \phi}}{\operatorname{argmax}} \mathcal{L}(q({\mathcal{Z}} \mid {\bf \phi})) \\
    &= \underset{{\bf \theta}, {\bf \phi}}{\operatorname{argmax}} \operatorname {E}_{q({\mathcal{Z}} = z \mid {\bf \phi})} \left[\log p(\mathcal{X} ,\mathcal{Z} = z \mid {\boldsymbol {\theta }})\right] - \operatorname{H}\left[\log q({\mathcal{Z}} \mid {\bf \phi})\right]  \\
\end{align}

In maximizing the ELBO, the first term, Expected Complete-data Log Likelihood, encourages the MLE / MAP estimates of the model parameters to be 

- 2. Find the posterior over the latent variables $\mathcal{Z}$, $p(\mathcal{Z} \mid \mathcal{X}, \boldsymbol {\theta_{\rm{max}} })$ {cite}`SVIPartI61:online`:

\begin{align}
    p(\mathcal{Z} \mid \mathcal{X}, \boldsymbol {\theta_{\rm{max}} }) &\approx q({\mathcal{Z}} \mid {\bf \phi}) \\
\end{align}

**Technique 1: Expectation-Maximization**

The EM algorithm seeks to find the MLE of the evidence / marginal likelihood / incomplete-data likelihood by iteratively applying these two steps {cite}`Expectat45:online`:

- 1. Expectation step (E step): Set the approximate posterior / variational distribution $q({\mathcal{Z}}\mid {\bf \phi}) = p(\mathcal{Z} \mid \mathcal{X}, \boldsymbol {\theta_{t} })$, where $\bf \theta_{t}$ are the previous M-step estimates of $\bf \theta$, this way the ${\rm KL}(q({\mathcal{Z}} \mid {\bf \phi}) \mid\mid p({\mathcal{Z}} \mid {\mathcal{x}}, {\bf \theta})) = 0$ and $\log p({\mathcal{x}} \mid {\bf \theta}) = \mathcal{L}(p({\mathcal{Z}} \mid {\mathcal{x}}, {\bf \theta_{t}}))$. Our objective is then to 

    - A. Calculate the posterior over latent variables $p(\mathcal{Z} \mid \mathcal{X} ,{\boldsymbol {\theta }}^{(t)})$ and 
    
    - B. Calculate $Q({\boldsymbol {\theta }}\mid {\boldsymbol {\theta }}^{(t)})$ (Expected Complete data Log Likelihood):

\begin{align}
    Q({\boldsymbol {\theta }}\mid {\boldsymbol {\theta }}^{(t)}) &= \operatorname {E} _{p(\mathcal{Z} = z \mid \mathcal{X} ,{\boldsymbol {\theta }}^{(t)})}\left[\log L({\boldsymbol {\theta }};\mathcal{X} ,\mathcal{Z} = z )\right]\, \\
    &= \operatorname {E} _{p(\mathcal{Z} = z \mid \mathcal{X} ,{\boldsymbol {\theta }}^{(t)})}\left[\log p(\mathcal{X} ,\mathcal{Z} = z \mid {\boldsymbol {\theta }}) \right]\, \\
    &= \sum_{z \in \mathcal{Z}} p(\mathcal{Z} = z \mid \mathcal{X} ,{\boldsymbol {\theta }}^{(t)}) \log p(\mathcal{X} ,\mathcal{Z} = z \mid {\boldsymbol {\theta }}) \\
\end{align}

Notice that the only thing that is missing from $Q({\boldsymbol {\theta }}\mid {\boldsymbol {\theta }}^{(t)})$ compared to the ELBO is the entropy of the approximate posterior distribution $\operatorname{H}\left[\log q({\mathcal{Z}} \mid {\bf \phi})\right]$.

- 2. Maximization step (M step): Find the parameters that maximize $ Q({\boldsymbol {\theta }}\mid {\boldsymbol {\theta }}^{(t)})$:

\begin{align} 
    {\boldsymbol {\theta }}^{(t+1)} &= {\underset {\boldsymbol {\theta }}{\operatorname {arg\,max} }}\ Q({\boldsymbol {\theta }}\mid {\boldsymbol {\theta }}^{(t)})\,
\end{align}

Example 01 - Gaussian Mixture Model {cite}`Expectat31:online`

Example 02 - Latent Dirichlet Allocation Topic Model

Example 03 - Hidden Markov Model {cite}`10.5555/1162264`{cite}`hmmbaumw42:online`

**HMM Setup**

Given the following setup:

- I.I.D. Observed data: $\mathcal{X} = \left(X^{(1)}, \cdots, X^{(D)}\right)$, 
    
    - where each $X^{(i)} = (x^{(i)}_{1}, x^{(i)}_{2}, \cdots, x^{(i)}_{T})$, 
    
    - where each $x^{(i)}_{t}$ can take on 1 of $\left{1, 2, \cdots K \right}$ values

- Unobserved latent data or missing values: $\mathcal{Z} = \left(Z^{(1)}, \cdots, Z^{(D)}\right)$, 
    
    - where each $Z^{(i)} = (z^{(i)}_{1}, z^{(i)}_{2}, \cdots, z^{(i)}_{T})$, 
    
    - where each $z^{(i)}_{t}$ can take on 1 of $\left{1, 2, \cdots N \right}$ values

- Unknown parameters: $\mathbf{\theta} = \left{A, B, \pi\right}$
    
    - where $\pi = (\pi_1, \pi_2, \cdots, \pi_N)$ is the $1 \times N$ initial state  matrix,
    
        - where each $\pi_{i} = p(z_{1} = i)$
    
    - where $A$ is the $N \times N$ state transmission matrix
    
        - where each $A_{ij} = p(z_{t+1} = j \mid z_{t} = i)$
    
    - where $B$ is the $N \times K$ state transmission matrix
    
        - where each $B_{i}(j) = p(x_{t} = j \mid z_{t} = i)$

**E-step**

\begin{align}
    
\end{align}

**Technique 2: Markov Chain Monte Carlo**

**Technique 3: Mean-field Approximation Variational Inference**

\begin{align}
    {\rm ELBO} &\equiv \mathbb{E}_{q({\bf z} \mid {\bf \phi})} \left [
\log p({\bf x}, {\bf z} \mid {\bf \theta}) - \log q({\bf z} \mid {\bf \phi})
\right]
\end{align}

**Example 01 - Gaussian Mixture Model**

**Example 02 - Latent Dirichlet Allocation Topic Model**

**Example 03 - Hidden Markov Model**

**Technique 4: Black-box Variational Inference**

\begin{align}
    {\rm ELBO} &\equiv \mathbb{E}_{q({\bf z} \mid {\bf \phi})} \left [
\log p({\bf x}, {\bf z} \mid {\bf \theta}) - \log q({\bf z} \mid {\bf \phi})
\right]
\end{align}

**Example 01 - Gaussian Mixture Model**

**Example 02 - Latent Dirichlet Allocation Topic Model**

**Example 03 - Hidden Markov Model**

**Example 04 - Bayesian Linear Regression**